# OpenAI Function-Calling Pipeline Demo

This notebook shows a **minimal, end-to-end tool-calling workflow** with OpenAI:
1. Two standalone Python tools (`add_numbers`, `to_upper`).
2. JSON tool definitions in the [function-calling format](https://platform.openai.com/docs/guides/function-calling).
3. A system prompt that tells the model *which* tool to call.
4. Code that routes a user prompt → model → tool execution → model again, returning the final answer.

> ⚠️  Set `OPENAI_API_KEY` in your environment before running.

In [54]:
from dotenv import load_dotenv
from enum import Enum
import json
import openai
from pydantic import BaseModel, Field
from typing import Dict, Any, Optional

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Make sure your API key is exported as an env var first:
#  export OPENAI_API_KEY="sk-..."
openai.api_key = OPENAI_API_KEY
assert openai.api_key, "OPENAI_API_KEY is not set"

client = openai.OpenAI(api_key=OPENAI_API_KEY)

In [55]:
# --- Context manager ------------------------------------------------
class Context:
    """A simple context manager to store and retrieve state between tool calls"""
    
    def __init__(self):
        self.data: Dict[str, Any] = {}
    
    def set(self, key: str, value: Any) -> None:
        """Store a value in the context"""
        self.data[key] = value
    
    def get(self, key: str, default: Any = None) -> Any:
        """Retrieve a value from the context"""
        return self.data.get(key, default)
    
    def update(self, values: Dict[str, Any]) -> None:
        """Update context with multiple values"""
        self.data.update(values)
    
    def __str__(self) -> str:
        return str(self.data)

In [52]:
# --- Tool names ---------------------------------------------------
class ToolName(Enum):
    ADD_NUMBERS = "add_numbers"
    TO_UPPER = "to_upper"

# --- Pydantic models for tool arguments --------------------------
class AddNumbersArgs(BaseModel):
    a: int = Field(..., description="The first integer")
    b: int = Field(..., description="The second integer")
    context: Optional[Dict[str, Any]] = Field(default=None, description="Optional context data")

class ToUpperArgs(BaseModel):
    text: str = Field(..., description="Text to convert")
    context: Optional[Dict[str, Any]] = Field(default=None, description="Optional context data")

In [49]:
# --- Tool definitions ---------------------------------------------------
tool_definitions = {
    ToolName.ADD_NUMBERS: {
        "name": ToolName.ADD_NUMBERS.value,
        "description": "Add two integers and return the sum.",
        "parameters": {
            "type": "object",
            "properties": {
                "a": {"type": "integer", "description": "The first integer"},
                "b": {"type": "integer", "description": "The second integer"}
            },
            "required": ["a", "b"]
        }
    },
    ToolName.TO_UPPER: {
        "name": ToolName.TO_UPPER.value,
        "description": "Convert text to UPPER-CASE.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {"type": "string", "description": "Text to convert"}
            },
            "required": ["text"]
        }
    }
}

In [53]:
# --- Tool handlers --------------------------------------------------------
def add_numbers(a: int, b: int, context: Context) -> int:
    """Add two integers and store the result in context"""
    result = a + b
    
    # Store the result and operands in context
    context.update({
        "last_operation": "add",
        "last_result": result,
        "operands": {"a": a, "b": b}
    })
    
    return result

def to_upper(text: str, context: Context) -> str:
    """Convert text to upper-case and store in context"""
    result = text.upper()
    
    # Store the input and result in context
    context.update({
        "last_operation": "to_upper",
        "last_result": result,
        "original_text": text
    })
    
    return result

tool_handlers = {
    ToolName.ADD_NUMBERS.value: add_numbers,
    ToolName.TO_UPPER.value: to_upper
}

In [45]:
# --- Tool names ---------------------------------------------------
class PromptName(Enum):
    DEFAULT = "default_prompt"
    ADD_NUMBERS = "add_numbers_prompt"
    TO_UPPER = "to_upper_prompt"

default_prompt = """You are a routing assistant.  
• If the user asks to *add* or *sum* numbers, call `add_numbers`.  
• If the user asks to transform text into upper-case, call `to_upper`.  
Respond only with a JSON tool call; no prose."""

prompts = {
    PromptName.DEFAULT: default_prompt
}

In [46]:
def run_pipeline(user_prompt: str, model: str = "gpt-4o-mini") -> str:
    """Route → tool → respond. Returns the assistant's final answer."""
    # 1️⃣ Ask the model which tool to invoke
    messages = [
        {"role": "system", "content": prompts[PromptName.DEFAULT]},
        {"role": "user", "content": user_prompt}
    ]
    
    # Convert the dictionary to a list for OpenAI API
    tool_definitions_list = list(tool_definitions.values())
    
    first = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tool_definitions_list,
        tool_choice="auto"
    )
    assistant_msg = first.choices[0].message
    tool_call = assistant_msg.tool_calls[0]

    # 2️⃣ Execute the tool locally
    name = tool_call.function.name
    args = json.loads(tool_call.function.arguments)
    
    # Use the tool_handlers dictionary to call the appropriate function
    if name in tool_handlers:
        # Create appropriate Pydantic model for args validation
        if name == ToolName.ADD_NUMBERS.value:
            args = AddNumbersArgs(**args_dict)
            result = tool_handlers[name](args, context)
        elif name == ToolName.TO_UPPER.value:
            args = ToUpperArgs(**args_dict)
            result = tool_handlers[name](args, context)
        else:
            raise ValueError(f"Handler exists but no Pydantic model for {name}")
    else:
        raise ValueError(f"Unknown tool {name}")

    # 3️⃣ Send the result back to the model
    tool_result_msg = {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "content": json.dumps({ "result": result })
    }

    follow_up = client.chat.completions.create(
        model=model,
        messages=messages + [assistant_msg, tool_result_msg]
    )
    
    return follow_up.choices[0].message.content

In [40]:
if __name__ == "__main__":
    print("Add 3 + 5  -->", run_pipeline("Please add 3 and 5."))
    print("Uppercase  -->", run_pipeline("make this uppercase: hello world"))

TypeError: keys must be str, int, float, bool or None, not ToolName